# ❗❗❗Внимание❗❗❗
### Данный ноутбук составлял участник нашей команды вне окружения проекта.

### Здесь используется нестандартное окружение, в связи с этим, вероятно возникнут трудности с запуском.

### Ноутбук предоставляется «как есть» и не несет важной составляющей инференса или обучения моделей.

### Для работы может потребоваться установка дополнительных библиотек и настройка окружения.

## Импорты

In [ ]:
import os
import base64
import sys

import torch
import numpy as np
import pandas as pd
import panel as pn
import shutil
from PIL import Image
from transformers import AutoModel
from huggingface_hub import hf_hub_download
from typing import Optional, Literal

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure
from torchvision.transforms import Compose, ToTensor, Normalize
import torchvision.transforms.functional as TF

import warnings
warnings.filterwarnings("ignore")

current_dir = os.getcwd()
parent_dir = os.path.join(current_dir, '..')
sys.path.append(parent_dir)
import scripts.utils.net as net

## Константы

In [ ]:
IMAGE_FOLDER = r"..\test_public\images" #путь к изображениям

EMBEDDINGS_PATH = r"emb_2d.npy" #путь к заранее посчитанным эмбеддингам изображений

HF_TOKEN = None

MODEL_PATH = r"..\ckpt_eer_epoch2_batch209000.ckpt" #путь к весам модели

## Переменные

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


aligner_path = os.path.expanduser('~/.cvlface_cache/minchul/cvlface_DFA_resnet50')
aligner_repo = 'minchul/cvlface_DFA_resnet50'

adaface_models = {
    'ir_101': MODEL_PATH
}
embedding_cache = {}

## Загрузка файла модели-выравнивателя из репозитория Hugging Face.

In [4]:
def download(repo_id: str, path: str, HF_TOKEN: Optional[str] = None) -> None:
    """
    :param repo_id: ID репозитория на Hugging Face.
    :param path: Локальный путь для сохранения файлов.
    :param HF_TOKEN: Токен аутентификации (если требуется).
    """
    os.makedirs(path, exist_ok=True)
    files_path = os.path.join(path, 'files.txt')
    
    if not os.path.exists(files_path):
        hf_hub_download(
            repo_id, 'files.txt', token=HF_TOKEN, 
            local_dir=path, local_dir_use_symlinks=False
        )
    
    with open(files_path, 'r', encoding='utf-8') as f:
        files = f.read().splitlines()
    
    additional_files = ['config.json', 'wrapper.py', 'model.safetensors']
    
    for file in [f for f in files if f] + additional_files:
        full_path = os.path.join(path, file)
        if not os.path.exists(full_path):
            hf_hub_download(
                repo_id, file, token=HF_TOKEN, 
                local_dir=path, local_dir_use_symlinks=False
            )

## Загрузка модели из локального пути

In [5]:
def load_model_from_local_path(path: str, HF_TOKEN: Optional[str] = None):
    """
    :param path: Путь к модели.
    :param HF_TOKEN: Токен аутентификации (если требуется).
    :return: Загруженная модель.
    """
    cwd = os.getcwd()
    os.chdir(path)
    sys.path.insert(0, path)
    
    model = AutoModel.from_pretrained(path, trust_remote_code=True, token=HF_TOKEN)
    
    os.chdir(cwd)
    sys.path.pop(0)
    return model

## Загрузка модели по идентификатору репозитория.

In [11]:
def load_model_by_repo_id(repo_id: str, save_path: str, HF_TOKEN: Optional[str] = None, force_download: bool = False):
    """
    :param repo_id: ID репозитория на Hugging Face.
    :param save_path: Путь для сохранения модели.
    :param HF_TOKEN: Токен аутентификации (если требуется).
    :param force_download: Принудительная загрузка (удаляет существующую директорию перед загрузкой).
    :return: Загруженная модель.
    """
    if force_download and os.path.exists(save_path):
        shutil.rmtree(save_path)
    
    download(repo_id, save_path, HF_TOKEN)
    return load_model_from_local_path(save_path, HF_TOKEN)

aligner = load_model_by_repo_id(aligner_repo, aligner_path, HF_TOKEN).to(device)

Loaded pretrained aligner from pretrained_model/model.pt


## Двойное выравнивание лица на изображении

In [12]:
def get_aligned_face(image_path: str) -> Image.Image:
    """
    :param image_path: Путь к изображению.
    :return: Выравненное изображение в формате PIL.Image.
    """
    trans = Compose([
        ToTensor(),
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    img = Image.open(image_path).convert('RGB')
    input_tensor = trans(img).unsqueeze(0).to(device)
    aligned_x, _, _, _, _, _ = aligner(input_tensor)
    aligned_x = (aligned_x[0] * 0.5 + 0.5).clamp(0, 1)
    aligned_pil = TF.to_pil_image(aligned_x)
    
    return aligned_pil

## Загрузка предобученной модели указанной архитектуры.

In [13]:
def load_pretrained_model(architecture: Literal['ir_101'] = 'ir_101'):
    """
    :param architecture: Название архитектуры модели (по умолчанию 'ir_101').
    :return: Загруженная и подготовленная к использованию модель.
    """
    assert architecture in adaface_models, f"Архитектура {architecture} не поддерживается."
    
    model_ = net.build_model(architecture)
    statedict = torch.load(
        adaface_models[architecture], map_location=torch.device('cpu')
    )['model_state_dict']
    
    model_.load_state_dict(statedict)
    model_.eval()
    
    return model_

## Загрузка состояния модели, оптимизатора и планировщика обучения из чекпоинта.

In [14]:
def load_checkpoint(
    filepath: str, 
    model: torch.nn.Module, 
    optimizer: Optional[torch.optim.Optimizer] = None, 
    scheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> torch.nn.Module:
    """
    :param filepath: Путь к файлу чекпоинта.
    :param model: Модель, в которую загружается состояние.
    :param optimizer: Опционально, оптимизатор для загрузки состояния.
    :param scheduler: Опционально, планировщик обучения для загрузки состояния.
    :return: Модель с загруженными весами.
    """
    checkpoint = torch.load(filepath, map_location=torch.device('cpu'))
    
    model.load_state_dict(checkpoint["model_state_dict"])
    
    if optimizer is not None and "optimizer_state_dict" in checkpoint:
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    
    if scheduler is not None and "scheduler_state_dict" in checkpoint:
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    
    epoch = checkpoint.get("epoch", -1)
    global_batch = checkpoint.get("global_batch", -1)
    
    print(f"Загружен чекпоинт: эпоха {epoch}, глобальный батч {global_batch}")
    
    return model

## Преобразование PIL-изображение (RGB) в тензор для модели AdaFace.

In [15]:
def to_input(pil_rgb_image: Image.Image) -> torch.Tensor:
    """
    :param pil_rgb_image: Входное изображение формата PIL (RGB).
    :return: Тензор изображения в формате BGR с нормализацией.
    """
    np_img = np.array(pil_rgb_image)
    bgr_img = np_img[:, :, ::-1]  # Преобразование RGB -> BGR
    bgr_img_norm = (bgr_img / 255.0 - 0.5) / 0.5  # Нормализация
    tensor = torch.tensor(bgr_img_norm.transpose(2, 0, 1), dtype=torch.float32)
    
    return tensor.unsqueeze(0)  # Добавление размерности батча

## Загрузка модели выравнивания, загрузка и перевод модели в режим оценки

In [ ]:
model = load_pretrained_model('ir_101').to(device)
model = load_checkpoint(
    "C:\\Users\\ilya\\Downloads\\new model\\ckpt_eer_epoch2_batch209000.ckpt", 
    model).to(device)
model.eval()

Loaded pretrained aligner from pretrained_model/model.pt
Загружен чекпоинт: эпоха 2, глобальный батч 209000


Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

## Вычисление эмбеддингов изображений с кэшированием результатов.

In [16]:
def get_embedding(image_path: str) -> Optional[torch.Tensor]:
    """
    :param image_path: Путь к изображению.
    :return: Тензор эмбеддинга или None в случае ошибки.
    """
    if image_path in embedding_cache:
        print(f"Эмбеддинг для {image_path} уже кэширован")
        return embedding_cache[image_path]
    
    try:
        aligned_img = get_aligned_face(image_path)
        if aligned_img is None:
            print(f"Ошибка при выравнивании изображения для {image_path}")
            return None
    except Exception as e:
        print(f"Ошибка при выравнивании изображения для {image_path}: {e}")
        return None
    
    print(f"Вычисляем эмбеддинг для {image_path}")
    tensor_input = to_input(aligned_img).to(device)
    
    with torch.no_grad():
        feature, _ = model(tensor_input)
    
    embedding_cache[image_path] = feature
    return feature

In [ ]:
if EMBEDDINGS_PATH is None:
    emb_list = []
    for root, _, files in os.walk(IMAGE_FOLDER):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):
                emb_list.append(get_embedding(os.path.join(root, file)))
    
    embeddings = torch.cat(emb_list, dim=0)
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)
else:
    embeddings_2d = np.load(EMBEDDINGS_PATH)

print(embeddings_2d)

[[  76.49864    76.17476 ]
 [  76.496445   76.172745]
 [  76.50054    76.17228 ]
 ...
 [  31.050276  -52.834854]
 [-151.27066   -45.050884]
 [  34.517803  -10.06106 ]]


## Получение пути ко всем изображениям

In [18]:
image_paths = []
for root, _, files in os.walk(IMAGE_FOLDER):
    for file in files:
        if file.endswith((".jpg", ".png", ".jpeg")):
            image_paths.append(os.path.join(root, file))
image_paths.sort()

## Формирорвание датафрейма с координатами и путями изображений

In [19]:
layout_images = []
for (x, y), img_path in zip(embeddings_2d, image_paths):
    layout_images.append({
        "img_path": img_path,
        "xref": "x",
        "yref": "y",
        "x": x,
        "y": y,
    })

pn.extension()

df = pd.DataFrame(layout_images)
df.head(3)

,img_path,xref,yref,x,y
0,C:\Users\ilya\Downloads\new model\data\test_pu...,x,y,76.498642,76.174759
1,C:\Users\ilya\Downloads\new model\data\test_pu...,x,y,76.496445,76.172745
2,C:\Users\ilya\Downloads\new model\data\test_pu...,x,y,76.500542,76.172279


## Преобразование изображения по заданному пути в data URL.

In [20]:
def path_to_data_url(path: str) -> str:
    """
    :param path: Путь к изображению.
    :return: Data URL изображения в формате base64.
    """
    with open(path, "rb") as f:
        data = f.read()
    encoded = base64.b64encode(data).decode("utf-8")
    
    return f"data:image/jpeg;base64,{encoded}"

# Применяем функцию
df["img_url"] = df["img_path"].apply(path_to_data_url)

## Кластеризация изображений изображения для упрощения набора фотографий

In [21]:
def cluster_images(df: pd.DataFrame, eps: float = 1.5, min_samples: int = 2) -> pd.DataFrame:
    """
    :param df: DataFrame с колонками "x", "y" и "img_url".
    :param eps: Параметр eps для DBSCAN (радиус соседства).
    :param min_samples: Минимальное количество точек в кластере.
    :return: DataFrame с центроидами кластеров и представительным изображением.
    """
    df_unique = df.drop_duplicates(subset=["x", "y"])
    coords = df_unique[["x", "y"]].values
    
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    df_unique["cluster"] = db.labels_
    
    clusters = df_unique[df_unique["cluster"] != -1].groupby("cluster")
    cluster_centers = []
    
    for cluster_id, group in clusters:
        center_x = group["x"].mean()
        center_y = group["y"].mean()
        representative_img = group.sample(1)["img_url"].values[0]  
        r_img_parh = group.sample(1)["img_path"].values[0]  
    
        cluster_centers.append({
            "x": center_x,
            "y": center_y,
            "img_url": representative_img,
            'img_path': r_img_parh
        })
    
    return pd.DataFrame(cluster_centers)

# Применение
cluster_df = cluster_images(df)

## Кластеризация точек в DataFrame и изменение размеров точек (w, h) для топ-кластеров в зависимости от расстояния до центра кластера.

In [ ]:
def adjust_cluster_point_sizes(
    cluster_df: pd.DataFrame,
    eps: float = 3.5,
    min_samples: int = 3,
    num_of_clusters: int = 35,
    default_size: float = 1.0,
    max_point_size: float = 10.0,
    min_point_size: float = 1.0
) -> pd.DataFrame:
    """
    Параметры:
        cluster_df (pd.DataFrame): DataFrame с координатами точек, должен содержать столбцы "x" и "y".
        eps (float): Параметр eps для алгоритма DBSCAN.
        min_samples (int): Минимальное количество точек для формирования кластера в DBSCAN.
        num_of_clusters (int): Количество топ-кластеров, для которых будут изменяться размеры точек.
        default_size (float): Размер точки по умолчанию для всех точек.
        max_point_size (float): Максимальный размер точки, применяемый для центра кластера.
        min_point_size (float): Минимальный размер точки, применяемый для краёв кластера.

    Возвращает:
        pd.DataFrame: Исходный DataFrame с добавленными столбцами "cluster", "w" и "h".
    """
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    cluster_df["cluster"] = dbscan.fit_predict(cluster_df[["x", "y"]])

    cluster_sizes = cluster_df[cluster_df["cluster"] != -1]["cluster"].value_counts()

    top_clusters = cluster_sizes.head(num_of_clusters).index.tolist()

    cluster_df["w"] = default_size
    cluster_df["h"] = default_size

    for cl in top_clusters:
        cluster_mask = cluster_df["cluster"] == cl
        center_x = cluster_df.loc[cluster_mask, "x"].mean()
        center_y = cluster_df.loc[cluster_mask, "y"].mean()

        distances = np.sqrt(
            (cluster_df.loc[cluster_mask, "x"] - center_x) ** 2 +
            (cluster_df.loc[cluster_mask, "y"] - center_y) ** 2
        )
        max_distance = distances.max()

        if max_distance > 0:
            normalized = distances / max_distance
        else:
            normalized = distances

        sizes = max_point_size - (max_point_size - min_point_size) * normalized

        cluster_df.loc[cluster_mask, "w"] = sizes
        cluster_df.loc[cluster_mask, "h"] = sizes

    return cluster_df

cluster_df = adjust_cluster_point_sizes(cluster_df)

## Визуализация результатов

In [23]:
source = ColumnDataSource(cluster_df)

x_min = cluster_df["x"].min() - 1
x_max = cluster_df["x"].max() + 1
y_min = cluster_df["y"].min() - 1
y_max = cluster_df["y"].max() + 1

p = figure(
    x_range=Range1d(x_min, x_max),
    y_range=Range1d(y_min, y_max),
    width=900,
    height=900,
    x_axis_type=None,
    y_axis_type=None,
)
p.output_backend = "webgl"

p.image_url(
    url="img_url",
    x="x",
    y="y",
    source=source,
    w="w",
    h="h",
    anchor="center"
)

layout = pn.Column(p)

In [21]:
layout

BokehModel(combine_events=True, render_bundle={'docs_json': {'91cc2f76-6633-4a3c-8581-b5fec6a6eb1e': {'version…

### Если график медленно прогружается, можно использовать визуализацию в браузере на localhost

In [24]:
pn.panel(layout).show()

Launching server at http://localhost:62547
